In [3]:
import math

In [4]:
def consensus(prof_mat):
    cons = ""
    for entry in prof_mat:
        max_pct = max(entry.values())
        for nuc, pct in entry.items():
            if pct == max_pct:
                cons += nuc
                max_pct = 0
    return cons

def gen_count_mat(dna_list, pseudo = True):
    ln = len(dna_list[0])
    count_mat = list()
    if pseudo:
        for i in range(ln):
            count_mat.append({"A":1, "C":1, "G":1, "T":1})
    else:
        for i in range(ln):
            count_mat.append({"A":0, "C":0, "G":0, "T":0})
    
    for i in range(len(dna_list[0])):
        for dna in dna_list:
            char = dna[i]
            count_mat[i][char] += 1
    
    return count_mat

def score_kmer(kmer, prof_mat):
    score = 1
    for i in range(len(kmer)):
        score *= prof_mat[i][kmer[i]]
    return score


def gen_prof_mat(dna_list, pseudo = True):
    count_mat = gen_count_mat(dna_list, pseudo)
    prof_mat = list()
    for _, slc_count in enumerate(count_mat):
        tot_sum = sum(slc_count.values())
        slc_prof = dict()
        for key, value in slc_count.items():
            slc_prof[key] = value / tot_sum
        prof_mat.append(slc_prof)
    return prof_mat


def gen_all_gapped_kmers(dna, k1, k2, g_vals):
    ln = len(dna)
    for g in g_vals:
#         print(g)
        for i in range(ln - (k1 + g + k2) - 1):
            slc = dna[i : i + g + k1 + k2]
#             print(slc)
#             print(slc[:k1] + slc[-k2:])
            yield slc[:k1] + slc[-k2:], g


def entropy(prof_mat):
    entrpy = 0
    for column in prof_mat:
        for nuc, val in column.items():
            if val != 0:
                entrpy += (val * math.log2(val))
    return -entrpy


def gen_prof_most_prob_gapped_kmer(dna, k1, k2, g_vals, prof_mat):
    most_prob_kmer = dna[:k1 + k2]
    best_score = score_kmer(most_prob_kmer, prof_mat)
    best_g = min(g_vals)
    for kmer, g in gen_all_gapped_kmers(dna, k1, k2, g_vals):
        curr_score = score_kmer(kmer, prof_mat)
        if curr_score > best_score:
            most_prob_kmer = kmer
            best_score = curr_score
            best_g = g
    return most_prob_kmer, best_g



# has a bug in storing the best g value, still need to track down
def greedy_gapped_motif_search(dna_list, k1, k2, t, g_vals):
    
    gmin = min(g_vals)
    best_motifs = [dna[:k1] + dna[gmin : k2 + gmin] for dna in dna_list]
    best_score = entropy(gen_prof_mat(best_motifs))
    # the g values are going awry, something is wrong
    best_gaps = [gmin for _ in range(len(dna_list))]
    
    for kmer, g in gen_all_gapped_kmers(dna_list[0], k1, k2, g_vals):
        
        gapped_motifs = [kmer[:k1] + kmer[-k2:]]
        
        # could be in the assignment or initialization of the current gaps matrix
        gaps = [0 for _ in range(len(dna_list))]
        gaps[0] = g
        
        for i in range(1, t):
            gapped_motif_prof = gen_prof_mat(gapped_motifs)
            
            # maybe it's not returning the correct values here?
            most_prob_kmer, most_prob_g = gen_prof_most_prob_gapped_kmer(dna_list[i], k1, k2,
                                                                         g_vals,
                                                                         gapped_motif_prof)
            
            gapped_motifs.append(most_prob_kmer)
            # could be here
            gaps[i] = most_prob_g
            
        curr_score = entropy(gen_prof_mat(gapped_motifs))
        if curr_score < best_score:
            best_score = curr_score
            best_motifs = gapped_motifs
            # probably not here, though could be shallow vs deep copy???
            best_gaps = gaps[:]
    
    return best_motifs, best_gaps


In [5]:
# Expect AGAG with gap lengths of 2, 3, 3, 2 in that order
test_strs = ["ATCGAGCTAGATTTA", "AGTTCAGACACACAC",
             "CAGATAGACGAGTTT", "ATAGACAGATAGTTT"]

k1 = 2
k2 = 2
t = len(test_strs)
g_vals = [2, 3, 4, 5]

greedy_gapped_motif_search(test_strs, k1, k2, t, g_vals)

(['AGAG', 'AGAG', 'AGAG', 'AGAG'], [2, 3, 2, 2])

## Heck yeah!
* We're getting the correct results!  That was super straightforward
* Still has a problem with returning the gap lengths, I'll get to that later
* Wait for Chien-Ju to make the test cases and get the length bug working

### Next steps
* We have larger test data cases, let's see how this algorithm works on them

In [67]:
with open("simulatedDna_n100_t50.txt") as f:
    med_test = [x.strip() for x in f.readlines()]

k1 = 7
k2 = 9
gmin = 11
gmax = 13
t = len(med_test)

med_gapped_motifs, med_best_g_vals = greedy_gapped_motif_search(med_test, k1, k2, t, gmin, gmax)

In [68]:
consensus(gen_prof_mat([x[:k1] for x in med_gapped_motifs]))

'CACAGTG'

In [69]:
consensus(gen_prof_mat([x[-k2:] for x in med_gapped_motifs]))

'ACAAAAACC'

### Kickass
* We can extract the motifs we're looking for
* Now let's see if this works on the big dataset
    * NOTE: Tried to run with gmin = 11, gmax = 24 and it took >90 minutes so it's just the smaller motif for now
    * Possible improvement: give an allowed list of gap values?

In [70]:
with open("simulatedDna_n600_t100.txt") as f:
    big_test = [x.strip() for x in f.readlines()]

k1 = 7
k2 = 9
gmin = 11
gmax = 13
t = len(big_test)

big_gapped_motifs, big_best_g_vals = greedy_gapped_motif_search(big_test, k1, k2, t, gmin, gmax)

In [71]:
consensus(gen_prof_mat([x[:k1] for x in big_gapped_motifs]))

'CACAGTG'

In [72]:
consensus(gen_prof_mat([x[-k2:] for x in big_gapped_motifs]))

'ACAAAAACC'